# Building a Custom Agent with Flotorch and Autogen

This notebook demonstrates how to build and run a simple AI agent using the Flotorch platform and the Autogen framework. The agent will be equipped with a custom tool to perform a specific function.

### Prerequesit
Configure model and API key in Flotroch console (https://console.flotorch.cloud/)

### Viewing logs
Logs can be viewed in logs tab in Flotroch console (https://console.flotorch.cloud/)

### Key Objectives:
- Install the required Flotorch autogen packages.
- Define a custom tool for a specific function using Autogen's `FunctionTool`.
- Configure a new agent with a specific role, goal, and tool.
- Utilize the Flotorch LLM to power the agent's reasoning capabilities.
- Assemble and execute a task with a team of agents.

## 1. Environment Setup and Imports

The following cells handle the complete setup of the environment. This includes:
1. **Installing** the necessary Flotorch ADK package.
2. **Importing** essential libraries from Flotorch and Autogen.
3. **Configuring** the API credentials and model endpoint.

In [ ]:
# install flotorch autogen package
%pip install flotorch[autogen]

In [ ]:
FLOTORCH_API_KEY = "<flotorch api key>"
FLOTORCH_BASE_URL = "<flotroch gateway base url>" # eg: https://gateway.flotorch.cloud"
FLOTORCH_MODEL = "<flotorch model>"

In [ ]:
# Import autogen and related modules
from flotorch.autogen.llm import FlotorchAutogenLLM
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_core.tools import FunctionTool
from autogen_agentchat.ui import Console


print("Imported necessary libraries successfully")

## 2. Defining a Custom Tool

To extend the agent's capabilities, we define a custom tool. The `FunctionTool` class from Autogen wraps a standard Python function, transforming it into a tool that the agent can utilize. This example creates a `get_weather` tool, which takes a city and optional units as input and returns a weather string. The function's docstring and the description provided in the `FunctionTool` helps the agent understand how and when to use it.

In [ ]:
def get_weather(city: str, units: str = "imperial") -> str:
    
        if units == "imperial":
            return f"The weather in {city} is 96 °F and Rainy."
        elif units == "metric":
            return f"The weather in {city} is 23 °C and Sunny."
        else:
            return f"Sorry, I don't know the weather in {city}."

tools = [FunctionTool(get_weather, "Get the weather for a given city.")]

print("Custom tool defined successfully.")

## 3. Configuring the Language Model

The agent requires a Large Language Model (LLM) to act as its brain. We instantiate `FlotorchAutogenLLM`, passing our API credentials and the desired model ID. This object serves as the bridge between our Autogen agent and the powerful reasoning capabilities of the Flotorch platform.

In [ ]:
model  =  FlotorchAutogenLLM(
        model_id=FLOTORCH_MODEL,
        api_key=FLOTORCH_API_KEY,
        base_url=FLOTORCH_BASE_URL,
    )

print(f"Flotorch LLM model configured")

## 4. Creating the Agent and Team

With the tool and LLM configured, we now define the agent's persona and assemble a team. We create an `AssistantAgent` with a clear `system_message` that defines its purpose and its access to the custom tool. We then place this agent into a `RoundRobinGroupChat` to handle the interactive conversation.

In [ ]:
assistant = AssistantAgent(
        name="FlotorchAssistant",
        model_client=model,
        tools = tools,
        system_message = """  
        You are a helpful assistant with access to one custom tool : 'get_weather' whenever user asks about the weather you need to use this tool,
         apart from that you can answer with your own knowledge with accurate answers.
        """ )
    
team = RoundRobinGroupChat(
        participants=[assistant],
        max_turns=1
    )
print(f"Agent and Task created successfully: {assistant.name}")

## 5. Interactive Chat

The final step is to run the interactive chat loop with the team. The `team.run_stream(task=user_input)` method starts the process, and the agent will autonomously use the provided tool based on the user's input to generate the final response.

In [ ]:
while True:
    user_input = input("user: ")

    if user_input.lower().strip() == "exit":
        break

    stream = team.run_stream(task = user_input)
    result = await Console(stream)


## Summary

This notebook demonstrated the end-to-end process of creating a functional AI agent using the Flotorch sdk and Autogen.

The key components included:

1. **Custom Tool Implementation**: A Python function was seamlessly converted into a usable tool for the agent using Autogen's `FunctionTool`.
2. **Model Configuration**: The `FlotorchAutogenLLM` was configured to provide the agent with its reasoning ability.
3. **Agent & Team Creation**: A specialized agent was defined with a clear system message and added to a `RoundRobinGroupChat` to ensure predictable behavior.
4. **Interactive Execution**: The agent and task were run in an interactive loop, successfully using the custom tool to produce the final output.
    
This example showcases the power and flexibility of the Flotorch platform for building customized, tool-augmented AI agents.